In [18]:
$raw = Import-Csv ./grace-bombs-raw.csv

$cleaned = $raw | % {
    $story = $_.'Tell us your Grace Bomb story'
    if ([string]::IsNullOrWhiteSpace($story)) {
        $story = $_.'share a little of what happened'
    }

    [pscustomobject]@{
        Name = $_.'Name'
        Email = $_.'Email'
        ReferralType = $_.'Howd you hear about Grace Bomb'
        ChurchOrRadioStation = $_.'Name of church or radio station'
        Date = $_.'Submitted On'
        Location = $_.'In what city and state'
        Type = $_.'Did you receive or drop a Grace Bomb'
        Story = $story
    }
}

In [ ]:
$locationCache = @{}

In [19]:
$searchLocationUri = 'https://nominatim.openstreetmap.org/search?format=json&q='
$InformationPreference = 'Continue'

$locationSearches = $cleaned | Select-Object *, @{
    l='LocationSearch'
    e={
        $location = $_.Location.ToLowerInvariant().Trim()

        if ([string]::IsNullOrWhiteSpace($location)) {
            [pscustomobject]@{
                OriginalLocation = $location
                FixedLocation = $null
                Latitude = $null
                Longitude = $null
                PlaceId = $null
            }
        }

        $response = $locationCache[$location]

        if ($response -eq $null) {
            Write-Information "Searching for $location..."
            $response = irm ($searchLocationUri + $location)

            if ($response -is [Array]) {
                $response = $response[0]
            }

            if ($response.display_name -ne $null) {
                $locationCache[$location] = $response
                Write-Information "... found ${$response.display_name}"
            } else {
                Write-Information "... nothing found"
            }
        }

        [pscustomobject]@{
            OriginalLocation = $location
            FixedLocation = $response.display_name
            Latitude = $response.lat
            Longitude = $response.lon
            PlaceId = $response.place_id
        }
    }
} | ? {$_.LocationSearch.FixedLocation -eq $null}

Searching for eugene, oregon...
... found 
Searching for petesa, american samoa...
... nothing found
Searching for turlock california...
... found 
Searching for panama new york...
... found 
Searching for ninety six sc...
... found 
Searching for owosso michigan...
... found 
Searching for milford mill, maryland...
... found 
Searching for baltimore  maryland...
... found 
Searching for ...
... nothing found


In [26]:
$locationSearches | ? {$_.LocationSearch.FixedLocation -ne $null} | select * -expand LocationSearch | select Date, FixedLocation, Latitude, Longitude,Story | convertto-json | Set-Clipboard

Select-Object: The property cannot be processed because the property "Name" already exists.
Select-Object: The property cannot be processed because the property "Email" already exists.
Select-Object: The property cannot be processed because the property "ReferralType" already exists.
Select-Object: The property cannot be processed because the property "ChurchOrRadioStation" already exists.
Select-Object: The property cannot be processed because the property "Date" already exists.
Select-Object: The property cannot be processed because the property "Location" already exists.
Select-Object: The property cannot be processed because the property "Type" already exists.
Select-Object: The property cannot be processed because the property "Story" already exists.
Select-Object: The property cannot be processed because the property "LocationSearch" already exists.
Select-Object: The property cannot be processed because the property "Name" already exists.
Select-Object: The property cannot be pr

Error: Command failed: SubmitCode: $locationSearches | ? {$_.LocationSearch.FixedLoca ...